In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading Data
pa = pd.read_csv('allegney_county_assesment.csv')

C:\Users\chosb\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (19,20,29,30,37,38,46,57,83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pa.head()

,PARID,PROPERTYHOUSENUM,PROPERTYFRACTION,PROPERTYADDRESS,PROPERTYCITY,PROPERTYSTATE,PROPERTYUNIT,PROPERTYZIP,MUNICODE,MUNIDESC,...,HALFBATHS,HEATINGCOOLING,HEATINGCOOLINGDESC,FIREPLACES,BSMTGARAGE,FINISHEDLIVINGAREA,CARDNUMBER,ALT_ID,TAXYEAR,ASOFDATE
0,0001G00217000000,100.0,,1ST AVE,PITTSBURGH,PA,,15222.0,101,1st Ward - PITTSBURGH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,01-APR-20
1,0001G00224000000,0.0,,FORT PITT BLVD,PITTSBURGH,PA,,15222.0,101,1st Ward - PITTSBURGH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,01-APR-20
2,0001G00224040100,151.0,,FORT PITT BLVD,PITTSBURGH,PA,UNIT 401,15222.0,101,1st Ward - PITTSBURGH,...,0.0,B,Central Heat with AC,NaN,NaN,1520.0,1.0,NaN,2020,01-APR-20
3,0001G00224040300,151.0,,FORT PITT BLVD,PITTSBURGH,PA,UNIT 403,15222.0,101,1st Ward - PITTSBURGH,...,0.0,B,Central Heat with AC,NaN,NaN,1858.0,1.0,NaN,2020,01-APR-20
4,0001G00224040400,151.0,,FORT PITT BLVD,PITTSBURGH,PA,UNIT 404,15222.0,101,1st Ward - PITTSBURGH,...,0.0,B,Central Heat with AC,NaN,NaN,1567.0,1.0,NaN,2020,01-APR-20


In [4]:
def one_hot_encode(df, column):
    """Implement one hot encoding on the DataFrame for the given column"""
    df = pd.concat([df, pd.get_dummies(df[column],prefix=column, dtype=int, dummy_na=True)], axis=1)
    df.drop([column],axis=1, inplace=True)
    return df
# NOTE: dummy_na=True treats NAN as a seperate Category.

# Drop SALEPRICE lower than 20,000 dollars
We only want to make predictions for existing SalesPrices that are a reasonable price.

In [5]:
pa = pa[pa['SALEPRICE'] > 20000]

# Verify nans are dropped.
print("Are there nulls in the SALEPRICE column? ", pa['SALEPRICE'].isnull().any())
print("The shape of the dataset is: ", pa.shape)

Are there nulls in the SALEPRICE column?  False
The shape of the dataset is:  (324493, 86)


# Grab Valid Sales
The SALECODE column tells us which Sales are valid, and which aren't. We want to grab the valid sales.

In [6]:
# Lowercase columns
pa['SALECODE'] = pa['SALECODE'].str.lower()

# Grab valid sales only
pa = pa[pa['SALECODE'].isin(['0', 'u', 'ur'])]

print("The shape of the dataset is: ", pa.shape)

The shape of the dataset is:  (166763, 86)


# Drop SALECODE and SALEDESC

In [7]:
pa = pa.drop(['SALEDESC'], axis=1)
pa = pa.drop(['SALECODE'], axis=1)

# Drop TAXCODE and one-hot-encode TAXDESC

In [8]:
pa = pa.drop(['TAXCODE'], axis=1)
pa = one_hot_encode(pa, 'TAXDESC')

# Drop TAXSUBCODE
Very minimal variance. Mostly `nan`

In [9]:
pa = pa.drop(['TAXSUBCODE'], axis=1)
pa = pa.drop(['TAXSUBCODE_DESC'], axis=1)

# One Hot Encode HOMESTEADFLAG
Either `HOM` or `nan`. Calling one_hot_encode will create a binary column.

In [10]:
pa = one_hot_encode(pa, 'HOMESTEADFLAG')

# One Hot Encode CLEANGREEN
Either `Y` or `nan`. Calling one_hot_encode will create a binary column

In [11]:
pa = one_hot_encode(pa, 'CLEANGREEN')

# Drop FARMSTEADFLAG
Very minimal variance. Mostly `nan` (Only 6 instances of `FRM`)

In [12]:
pa = pa.drop(['FARMSTEADFLAG'], axis=1)

# One Hot Encode ABATEMENTFLAG
Either `Y` or `nan`

In [13]:
pa = one_hot_encode(pa, 'ABATEMENTFLAG')

# Binning PREVSALEPRICE
Bin PREVSALEPRICES. `NAN` columns won't be put into a bin (ie. all columns for PREVSALEPRICE will be 0)

In [14]:
print(f"There are {pa['PREVSALEPRICE'].isna().sum()} null previous Sale Prices")

There are 61487 null previous Sale Prices


In [15]:
pa['PREVSALEPRICE'] = pd.qcut(pa['PREVSALEPRICE'], 15, duplicates='drop')
pa = one_hot_encode(pa, 'PREVSALEPRICE')

# Binning PREVSALEPRICE2
Bin PREVSALEPRICES2. `NAN` columns won't be put into a bin (ie. all columns for PREVSALEPRICE2 will be 0)

In [16]:
pa['PREVSALEPRICE2'] = pd.qcut(pa['PREVSALEPRICE2'], 15, duplicates='drop')
pa = one_hot_encode(pa, 'PREVSALEPRICE2')

# We will keep COUNTYBUILDING AS IS
This is a continuous feature that is clean and has no NULLs

In [17]:
pa['COUNTYBUILDING'].isnull().any()

False

# We will keep COUNTYLAND AS IS
This is a continuous feature that is clean and has no NULLs

In [18]:
pa['COUNTYLAND'].isnull().any()

False

# We will keep COUNTYTOTAL AS IS
This is a continuous feature that is clean and has no NULLs

In [19]:
pa['COUNTYTOTAL'].isnull().any()

False

# Drop COUNTYEXEMPTBLDG
Almost all 0. 

In [20]:
pa = pa.drop(['COUNTYEXEMPTBLDG'], axis=1)

# We will keep LOCALBUILDING as is
This is a continuous feature that is clean and has no NULLs

In [21]:
pa['LOCALBUILDING'].isna().sum()

0

# We will keep LOCALLAND as is
This is a continuous feature that is clean and has no NULLs

In [22]:
pa['LOCALLAND'].isna().sum()

0

# We will keep FAIRMARKETBUILDING as is
This is a continuous feature that is clean and has no NULLs

In [23]:
pa['FAIRMARKETBUILDING'].isna().sum()

0

# We will keep FAIRMARKETLAND as is
This is a continuous feature that is clean and has no NULLs

In [24]:
pa['FAIRMARKETLAND'].isna().sum()

0

# We will keep FAIRMARKETTOTAL as is
This is a continuous feature that is clean and has no NULLs

In [25]:
pa['FAIRMARKETTOTAL'].isna().sum()

0

# Drop OWNERCODE and one-hot-encode OWNERDESC

In [26]:
pa = pa.drop(['OWNERCODE'], axis=1)
pa = one_hot_encode(pa, 'OWNERDESC')

# Drop CLASS One Hot Encode CLASS_DESC

In [27]:
pa = pa.drop(['CLASS'], axis=1)
pa = one_hot_encode(pa, 'CLASSDESC')

# Drop STYLE and One Hot Encode STYLEDESC

In [28]:
pa = pa.drop(['STYLE'], axis=1)
pa = one_hot_encode(pa, 'STYLEDESC')

# Clean STORIES
* Round to nearest half

* Impute nans with average

* One Hot Encode as they number of stories is still categorical

In [29]:
# Round stories to the nearest half.
pa['STORIES'] = round(pa['STORIES'] * 2) / 2

# Fill NANs with mean stories.
mean_stories = round(pa['STORIES'].mean() * 2) / 2
pa['STORIES'] = pa['STORIES'].fillna(value=mean_stories)

pa = one_hot_encode(pa, 'STORIES')

# Map Year to Reference Year
* Additionally set NAN's to avg year

In [30]:
pa['YEARBLT'] = 2020 - pa['YEARBLT']
pa['YEARBLT'] = pa['YEARBLT'].fillna(round(pa['YEARBLT'].mean()))

# Drop EXTERIORFINISH and One Hot Encode EXTFINISH_DESC

In [31]:
pa = pa.drop(['EXTERIORFINISH'], axis=1)
pa = one_hot_encode(pa, 'EXTFINISH_DESC')

# Drop ROOF and One Hot Encode ROOFDESC

In [32]:
pa = pa.drop(['ROOF'], axis=1)
pa = one_hot_encode(pa, 'ROOFDESC')

# Drop BASEMENT and One Hot Encode BASEMENTDESC

In [33]:
pa = pa.drop(['BASEMENT'], axis=1)
pa = one_hot_encode(pa, 'BASEMENTDESC')

# One Hot Encode GRADE

In [34]:
pa = pa.drop(['GRADE'], axis=1)
pa = one_hot_encode(pa, 'GRADEDESC')

# Drop ROOF and One Hot Encode ROOFDESC

In [35]:
pa = pa.drop(['CONDITION'], axis=1)
pa = one_hot_encode(pa, 'CONDITIONDESC')

# One Hot Encode CDU

In [36]:
pa = pa.drop(['CDU'], axis=1)
pa = one_hot_encode(pa, 'CDUDESC')

# TOTALROOMS
* Fill NANs with average

Keep as is.

In [37]:
pa['TOTALROOMS'] = pa['TOTALROOMS'].fillna(value=round(pa['TOTALROOMS'].mean())) 

# BEDROOMS
* Fill NANs with average

Keep as is.

In [38]:
pa['BEDROOMS'] = pa['BEDROOMS'].fillna(value=round(pa['BEDROOMS'].mean())) 

# FULLBATHS
* Fill NANs with average

Keep as is.

In [39]:
pa['FULLBATHS'] = pa['FULLBATHS'].fillna(value=round(pa['FULLBATHS'].mean())) 

# HALFBATHS
* Fill NANs with average

Keep as is.

In [40]:
pa['HALFBATHS'] = pa['HALFBATHS'].fillna(value=round(pa['HALFBATHS'].mean())) 

# One Hot Encode HEATINGCOOLING

In [41]:
pa = pa.drop(['HEATINGCOOLING'], axis=1)
pa = one_hot_encode(pa, 'HEATINGCOOLINGDESC')

# FIREPLACES
* Fill NANs with average

Keep as is.

In [42]:
pa['FIREPLACES'] = pa['FIREPLACES'].fillna(value=round(pa['FIREPLACES'].mean())) 

# BSMTGARAGE
* Fill NANs with average

Keep as is.

In [43]:
pa['BSMTGARAGE'] = pa['BSMTGARAGE'].fillna(value=round(pa['BSMTGARAGE'].mean())) 

# FINISHEDLIVINGAREA
* Fill NANs with average.

Keep as is.

In [44]:
pa['FINISHEDLIVINGAREA'] = pa['FINISHEDLIVINGAREA'].fillna(value=round(pa['FINISHEDLIVINGAREA'].mean())) 

# Dropping MUNIDESC and MUNICODE
These values seem to have limited value (maybe?) Dropping for now.

In [45]:
pa = pa.drop(['MUNIDESC'], axis=1)
pa = pa.drop(['MUNICODE'], axis=1)

# Drop SCHOOLCODE

School code seems like it would be predictive, but it's too large for us to label as categorical. Not sure what to do here. Dropping for now.

In [46]:
pa = pa.drop(['SCHOOLCODE'], axis=1)
pa = pa.drop(['SCHOOLDESC'], axis=1)

# DROP NEIGHCODE
Same sentiment as SCHOOLCODE. Seems valuable, but not sure how to effectively capture the info.

In [47]:
pa = pa.drop(['NEIGHCODE'], axis=1)
pa = pa.drop(['NEIGHDESC'], axis=1)

# LOTAREA
* Fill NANs with average

Keep as is.

In [48]:
pa['LOTAREA'] = pa['LOTAREA'].fillna(value=round(pa['LOTAREA'].mean())) 

# Drop USECODE and USEDESC
These are more specific than Class. We'll drop them for now as they add a lot of dimensionality.

In [49]:
pa = pa.drop(['USECODE'], axis=1)
pa = pa.drop(['USEDESC'], axis=1)

# Drop RECORDDATE and SALEDATE 
Dropping for now... We may want to create a reference date that we can map each date to.

In [50]:
pa = pa.drop(['RECORDDATE'], axis=1)
pa = pa.drop(['SALEDATE'], axis=1)

# Dropping Unneeded Columns


In [51]:
pa = pa.drop(['DEEDBOOK', 'DEEDPAGE'], axis=1) # DEED
pa = pa.drop(['CHANGENOTICEADDRESS1', 'CHANGENOTICEADDRESS2', 'CHANGENOTICEADDRESS3', 'CHANGENOTICEADDRESS4'], axis=1) # ADDRESS
pa = pa.drop(['CARDNUMBER', 'ALT_ID', 'ASOFDATE', 'TAXYEAR'], axis=1) # EXTRA
pa = pa.drop(['LEGAL1', 'LEGAL2', 'LEGAL3'], axis=1) # LEGAL
pa = pa.drop(['PROPERTYUNIT', 'PROPERTYSTATE', 'PROPERTYADDRESS', 'PROPERTYHOUSENUM', 'PARID'], axis=1) # LEGAL
pa = pa.drop(['PROPERTYFRACTION', 'PROPERTYCITY'], axis=1)
pa = pa.drop(['PREVSALEDATE', 'PREVSALEDATE2'], axis=1)

In [52]:
pa.columns.tolist()

['PROPERTYZIP',
 'LOTAREA',
 'SALEPRICE',
 'COUNTYBUILDING',
 'COUNTYLAND',
 'COUNTYTOTAL',
 'LOCALBUILDING',
 'LOCALLAND',
 'LOCALTOTAL',
 'FAIRMARKETBUILDING',
 'FAIRMARKETLAND',
 'FAIRMARKETTOTAL',
 'YEARBLT',
 'TOTALROOMS',
 'BEDROOMS',
 'FULLBATHS',
 'HALFBATHS',
 'FIREPLACES',
 'BSMTGARAGE',
 'FINISHEDLIVINGAREA',
 'TAXDESC_10 - Exempt',
 'TAXDESC_12 - Public Utility Realty Tax - PURTA',
 'TAXDESC_20 - Taxable',
 'TAXDESC_nan',
 'HOMESTEADFLAG_HOM',
 'HOMESTEADFLAG_nan',
 'CLEANGREEN_Y',
 'CLEANGREEN_nan',
 'ABATEMENTFLAG_Y',
 'ABATEMENTFLAG_nan',
 'PREVSALEPRICE_(-0.001, 1.0]',
 'PREVSALEPRICE_(1.0, 17000.0]',
 'PREVSALEPRICE_(17000.0, 36000.0]',
 'PREVSALEPRICE_(36000.0, 51456.0]',
 'PREVSALEPRICE_(51456.0, 66500.0]',
 'PREVSALEPRICE_(66500.0, 83000.0]',
 'PREVSALEPRICE_(83000.0, 105000.0]',
 'PREVSALEPRICE_(105000.0, 131700.0]',
 'PREVSALEPRICE_(131700.0, 170000.0]',
 'PREVSALEPRICE_(170000.0, 255000.0]',
 'PREVSALEPRICE_(255000.0, 55000000.0]',
 'PREVSALEPRICE_nan',
 'PREVSAL

In [53]:
pa.to_csv("preprocess.csv", index=False)

In [62]:
pa['PROPERTYZIP'].value_counts()

15237.0    7235
15108.0    6169
15235.0    6017
15102.0    5270
15236.0    4885
           ... 
16229.0       5
15083.0       4
15089.0       2
16056.0       1
15012.0       1
Name: PROPERTYZIP, Length: 117, dtype: int64